# Project 1: Accommodation Prices Prediction - Data Cleaning

After scraping relevant information about the stays on Airbnb and Booking.com, we can move on and combine the two datasets into one. Although both datasets contain the same set of variables, their formats are somewhat different. Therefore, data cleaning is required to align these variables.

Import Both datasets

In [4]:
import pandas as pd
import numpy as np

df_airbnb = pd.read_csv('../scraping/airbnb_tokyo.csv')
df_hotel = pd.read_csv('../scraping/hotel_tokyo.csv')

In [5]:
df_airbnb.head()

,Description,Number of Beds,Price (HKD)
0,Hotel room in Chuo City,1 small double bed,"$37 \n$32\n night\n$32 per night, originally $37"
1,Private room in Sumida-ku,1 double bed,"$40 \n$23\n night\n$23 per night, originally $40"
2,Hotel room in 中央区,3 beds,"$160 \n$62\n night\n$62 per night, originally ..."
3,Hotel room in Chuo City,1 double bed,"$45 \n$36\n night\n$36 per night, originally $45"
4,Apartment in Chuo City,1 bed,"$79 \n$55\n night\n$55 per night, originally $79"


In [6]:
df_hotel.head()

,Number of Beds,Location,Price (HKD)
0,Entire apartment • 1 bedroom • 1 bathroom • 17...,"Kita, Tokyo",HK$ 589
1,"2 beds (1 double, 1 futon)","Taito, Tokyo",HK$ 621
2,Entire apartment • 1 bedroom • 1 living room •...,"Shinagawa Ward, Tokyo","HK$ 1,807"
3,Entire apartment • 1 bedroom • 1 bathroom\n2 b...,"Sumida Ward, Tokyo",HK$ 811
4,1 single bed,"Taito, Tokyo",HK$ 378


From above we can see that both datasets contain the price, location, and the number of beds for each stay, but they are presented in different formats. Also, both datasets actually contains the types of the stay e.g. hotel room, apartment, but they are hidden in different variables. Our task is to separate them out and make it a new variable.

### cleaning the airbnb dataset

In [7]:
df_airbnb[['Type', 'Location']] = df_airbnb['Description'].str.split(' in ', expand=True)
# df_airbnb['Type of Beds'] = df_airbnb['Number of Beds'].str.replace(' bed', '').str.split(' ', 1, expand=True)[1].fillna('typical')
df_airbnb['Number of Beds'] = df_airbnb['Number of Beds'].str.split(' ', expand=True)[0]
df_airbnb['Number of Beds'] = df_airbnb['Number of Beds'].replace('', 0)
df_airbnb['Price (HKD)'] = df_airbnb['Price (HKD)'].str.split(' ', expand=True)[0].str.split('$', expand=True)[1].str.replace(',', '')
df_airbnb = df_airbnb.drop(columns='Description')
df_airbnb['Number of Beds'] = df_airbnb['Number of Beds'].astype('float64')
df_airbnb['Price (HKD)'] = df_airbnb['Price (HKD)'].astype('float64')

df_airbnb.head()

,Number of Beds,Price (HKD),Type,Location
0,1.0,37.0,Hotel room,Chuo City
1,1.0,40.0,Private room,Sumida-ku
2,3.0,160.0,Hotel room,中央区
3,1.0,45.0,Hotel room,Chuo City
4,1.0,79.0,Apartment,Chuo City


In [8]:
df_hotel.head()

,Number of Beds,Location,Price (HKD)
0,Entire apartment • 1 bedroom • 1 bathroom • 17...,"Kita, Tokyo",HK$ 589
1,"2 beds (1 double, 1 futon)","Taito, Tokyo",HK$ 621
2,Entire apartment • 1 bedroom • 1 living room •...,"Shinagawa Ward, Tokyo","HK$ 1,807"
3,Entire apartment • 1 bedroom • 1 bathroom\n2 b...,"Sumida Ward, Tokyo",HK$ 811
4,1 single bed,"Taito, Tokyo",HK$ 378


### Cleaning the Hotel Dataset

In [9]:
# cleaning the hotel dataset
df_hotel['Price (HKD)'] = df_hotel['Price (HKD)'].str.split(' ', expand=True)[1].str.replace(',', '').astype('float64')
df_hotel['Location'] = df_hotel['Location'].str.split(', ', expand=True)[0]

Next, we categorize the types of stays, which could be tricky since the descriptions are not structured the same way.

In [10]:
bedtypes = df_hotel['Number of Beds'].value_counts()

Find the patterns of the description

In [11]:
bedtypes.to_csv('bedtypes.csv')

In [12]:
df_hotel_types = df_hotel['Number of Beds'].copy()

In [13]:
df_hotel_types.value_counts()

1 single bed                                                                                                            397
1 double bed                                                                                                            231
1 large double bed                                                                                                       57
Bed in dormitory\n1 bunk bed                                                                                             46
2 single beds                                                                                                            43
                                                                                                                       ... 
Entire apartment • 1 bedroom • 1 living room • 1 bathroom • 1 kitchen • 25m²\n3 beds (1 single, 1 sofa bed, 1 futon)      1
5 beds (2 singles, 3 sofa beds)                                                                                           1
Entire s

In [14]:
df_hotel_types[df_hotel_types.str.contains('apartment', na=False)] = 'Apartment'
df_hotel_types[df_hotel_types.str.contains('Entire villa', na=False)] = 'Villa'
df_hotel_types[df_hotel_types.str.contains('studio', na=False)] = 'Studio'
df_hotel_types[df_hotel_types.str.contains('suite', na=False)] = 'Suite'
df_hotel_types[df_hotel_types.str.contains('Entire holiday home', na=False)] = 'Holiday Home'

In [15]:
df_hotel_types[df_hotel_types.str.contains('dormitory', na=False)] = 'Dormitory'

In [16]:
df_hotel_types[(df_hotel_types.str.contains('Beds:', na=False)) | df_hotel_types.str.contains('Multiple', na=False)] = 'Miscellaneous'

In [17]:
non_hotel_rooms = ['Apartment', 'Studio', 'Villa', 'Holiday Home', 'Dormitory', 'Miscellaneous', 'Suite']
df_hotel_types[~df_hotel_types.str.contains('|'.join(non_hotel_rooms), na=False)] = 'Hotel Room'

In [18]:
df_hotel_types.value_counts()

Hotel Room       846
Dormitory         86
Apartment         40
Studio            15
Miscellaneous      4
Suite              4
Villa              3
Holiday Home       2
Name: Number of Beds, dtype: int64

In [19]:
df_hotel_types.value_counts().sum()

1000

In [20]:
df_hotel['Type'] = df_hotel_types

In [21]:
df_hotel.head()

,Number of Beds,Location,Price (HKD),Type
0,Entire apartment • 1 bedroom • 1 bathroom • 17...,Kita,589.0,Apartment
1,"2 beds (1 double, 1 futon)",Taito,621.0,Hotel Room
2,Entire apartment • 1 bedroom • 1 living room •...,Shinagawa Ward,1807.0,Apartment
3,Entire apartment • 1 bedroom • 1 bathroom\n2 b...,Sumida Ward,811.0,Apartment
4,1 single bed,Taito,378.0,Hotel Room


We succesfully classified the stays of the hotel dataset. Then we can turn to the extraction of the number of beds. The formats of the descriptions are different depdending on the type of stay. Therefore, we have to extract them accordingly.

In [22]:
df_hotel_copy = df_hotel.copy()
df_num_rooms = df_hotel_copy['Number of Beds'][df_hotel_copy['Type'] == 'Hotel Room'].str.split(' ', expand=True)[0]
df_num_rooms = df_num_rooms.append(df_hotel_copy[df_hotel_copy['Type'] == 'Dormitory']['Number of Beds'].str.split('\n', expand=True)[1].str.split(' ', expand=True)[0])
df_num_rooms = df_num_rooms.sort_index()

In [23]:
df_hotel = pd.concat((df_hotel, df_num_rooms), axis=1)
df_hotel = df_hotel.rename(columns={0: 'Beds'})

In [24]:
df_hotel

,Number of Beds,Location,Price (HKD),Type,Beds
0,Entire apartment • 1 bedroom • 1 bathroom • 17...,Kita,589.0,Apartment,NaN
1,"2 beds (1 double, 1 futon)",Taito,621.0,Hotel Room,2
2,Entire apartment • 1 bedroom • 1 living room •...,Shinagawa Ward,1807.0,Apartment,NaN
3,Entire apartment • 1 bedroom • 1 bathroom\n2 b...,Sumida Ward,811.0,Apartment,NaN
4,1 single bed,Taito,378.0,Hotel Room,1
...,...,...,...,...,...
995,1 single bed,Ota Ward,790.0,Hotel Room,1
996,1 single bed,Toshima,364.0,Hotel Room,1
997,2 double beds,Toshima,683.0,Hotel Room,2
998,Entire apartment • 1 bedroom • 1 bathroom • 1 ...,Shibuya Ward,1298.0,Apartment,NaN


In [25]:
df_hotel = df_hotel.drop(columns='Number of Beds')
df_hotel = df_hotel.rename(columns={'Beds': 'Number of Beds'})

In [26]:
df_hotel

,Location,Price (HKD),Type,Number of Beds
0,Kita,589.0,Apartment,NaN
1,Taito,621.0,Hotel Room,2
2,Shinagawa Ward,1807.0,Apartment,NaN
3,Sumida Ward,811.0,Apartment,NaN
4,Taito,378.0,Hotel Room,1
...,...,...,...,...
995,Ota Ward,790.0,Hotel Room,1
996,Toshima,364.0,Hotel Room,1
997,Toshima,683.0,Hotel Room,2
998,Shibuya Ward,1298.0,Apartment,NaN


In [27]:
df_airbnb

,Number of Beds,Price (HKD),Type,Location
0,1.0,37.0,Hotel room,Chuo City
1,1.0,40.0,Private room,Sumida-ku
2,3.0,160.0,Hotel room,中央区
3,1.0,45.0,Hotel room,Chuo City
4,1.0,79.0,Apartment,Chuo City
...,...,...,...,...
295,6.0,1445.0,Home,豊島区
296,2.0,255.0,Apartment,Katsushika City
297,2.0,292.0,Private room,Narita
298,1.0,963.0,Home,Taito City


After cleaning the Airbnb and the hotel datasets individually, we can finally combine the two while keeping track of the values of each of the variables.

In [28]:
# Rearranging the columns
df_hotel = df_hotel.reindex(columns=['Location', 'Number of Beds', 'Type', 'Price (HKD)'])
df_airbnb = df_airbnb.reindex(columns=['Location', 'Number of Beds', 'Type', 'Price (HKD)'])

Let's first clean the 'Location' variable by grouping the values into the 23 wards i.e. districts in Tokyo.

In [29]:
airbnb_location = df_airbnb['Location'].value_counts()
airbnb_location.to_csv('airbnb_location.csv')

In [30]:
hotel_location = df_hotel['Location'].value_counts()
hotel_location.to_csv('hotel_location.csv')

In [31]:
airbnb_location_labelled = pd.read_csv('airbnb_location_labelled.csv', names=['Original Location', 'New Location'])
hotel_location_labelled = pd.read_csv('hotel_location_labelled.csv', names=['Original Location', 'New Location'])

In [32]:
# check if number of unqiue values in each dataset exceeds 24
print(len(df_airbnb['Location'].replace(airbnb_location_labelled['Original Location'].values, airbnb_location_labelled['New Location'].values).value_counts()))
print(len(df_hotel['Location'].replace(hotel_location_labelled['Original Location'].values, hotel_location_labelled['New Location'].values).value_counts()))

22
24


In [33]:
df_airbnb['Location'] = df_airbnb['Location'].replace(airbnb_location_labelled['Original Location'].values, airbnb_location_labelled['New Location'].values)
df_hotel['Location'] = df_hotel['Location'].replace(hotel_location_labelled['Original Location'].values, hotel_location_labelled['New Location'].values)

### Types of Stay

In [34]:
df_airbnb['Type'] = df_airbnb['Type'].replace(['Hotel room', 'Vacation home'], ['Hotel Room', 'Holiday Home'])

### Combining Two Datasets

In [35]:
airbnb_label = pd.Series(['Airbnb' for _ in range(0,len(df_airbnb))])
hotel_label = pd.Series(['Hotel' for _ in range(0,len(df_hotel))])

In [36]:
df_airbnb = pd.concat((df_airbnb, airbnb_label), axis=1)
df_hotel = pd.concat((df_hotel, hotel_label), axis=1)
df_airbnb = df_airbnb.rename(columns={0:'Source'})
df_hotel = df_hotel.rename(columns={0:'Source'})

In [37]:
df_acm = pd.concat((df_hotel, df_airbnb), ignore_index=True)

In [38]:
df_acm = df_acm.reindex(columns=['Source', 'Location', 'Number of Beds', 'Type', 'Price (HKD)'])

In [39]:
df_acm.head()

,Source,Location,Number of Beds,Type,Price (HKD)
0,Hotel,Kita,NaN,Apartment,589.0
1,Hotel,Taito,2,Hotel Room,621.0
2,Hotel,Shinagawa,NaN,Apartment,1807.0
3,Hotel,Sumida,NaN,Apartment,811.0
4,Hotel,Taito,1,Hotel Room,378.0


In [40]:
df_acm.shape

(1300, 5)

In [41]:
df_acm['Number of Beds'] = df_acm['Number of Beds'].astype('float64')

In [42]:
df_acm.to_csv('accommodation.csv', sep=',', index=False)